### Derivation of Mean Average Precision for the VOC model
Hyper parameters:
* Two cells in each row and column.
* Two predicted bounding boxes for each cell.
* Three object classes.
* Two batches.

In [389]:
import torch
import torch.nn as nn

In [390]:
BATCHES = 2
CELLS = 2
CLASSES = 3
BOXES = 2
LABEL_ITEM_SIZE = CLASSES + 5
PRED_ITEM_SIZE = CLASSES + 5 * BOXES

IOU_THRESHOLDS = [0.5, 0.75]

In [391]:
# create sample prediction label
label = torch.zeros((BATCHES, CELLS, CELLS, LABEL_ITEM_SIZE))

# batch 0: cell: (0, 0), class: 0, location in the middle
label[0][0][0] = torch.Tensor([1, 0, 0, 1, 0.5, 0.5, 0.9, 0.9])

# batch 0: cell: (1, 0), class: 0, location in the middle
label[0][1][0] = torch.Tensor([1, 0, 0, 1, 0.51, 0.49, 0.6, 0.6])

# batch 1: cell: (1, 1), class: 1, location in the middle
label[1][1][1] = torch.Tensor([0, 1, 0, 1, 0.5, 0.5, 0.8, 0.8])

In [392]:
# create sample prediction data
pred = torch.rand((BATCHES, CELLS, CELLS, PRED_ITEM_SIZE)) * 0.1

# batch 0: cell: (0, 0), box: 0, class: 0, true positive
pred[0][0][0][:CLASSES] = torch.Tensor([0.8, 0.2, 0.5])
pred[0][0][0][CLASSES: CLASSES + 5] = torch.Tensor([0.9, 0.45, 0.55, 0.85, 0.95])

# batch 1: cell: (1, 1), box: 1, class: 1, true positive
pred[1][1][1][:CLASSES] = torch.Tensor([0.1, 0.7, 0.1])
pred[1][1][1][CLASSES + 5: CLASSES + 10] = torch.Tensor([0.6, 0.44, 0.52, 0.8, 0.8])

# batch 1: cell: (0, 1), box: 0, class: 2, false positive
pred[1][0][0][:CLASSES] = torch.Tensor([0.1, 0.5, 0.8])
pred[1][0][0][CLASSES: CLASSES + 5] = torch.Tensor([0.7, 0.1, 0.1, 1.0, 0.2])

In [393]:
print('labels')
for i in range(BATCHES):
	print(f'batch {i}')
	print(label[i])

labels
batch 0
tensor([[[1.0000, 0.0000, 0.0000, 1.0000, 0.5000, 0.5000, 0.9000, 0.9000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[1.0000, 0.0000, 0.0000, 1.0000, 0.5100, 0.4900, 0.6000, 0.6000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])
batch 1
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0000, 0.0000, 1.0000, 0.5000, 0.5000, 0.8000, 0.8000]]])


In [394]:
print('predictions')
for i in range(BATCHES):
	print(f'batch {i}')
	print(pred[i])

predictions
batch 0
tensor([[[0.8000, 0.2000, 0.5000, 0.9000, 0.4500, 0.5500, 0.8500, 0.9500,
          0.0948, 0.0910, 0.0930, 0.0036, 0.0175],
         [0.0931, 0.0965, 0.0706, 0.0945, 0.0326, 0.0823, 0.0730, 0.0206,
          0.0991, 0.0570, 0.0791, 0.0298, 0.0918]],

        [[0.0863, 0.0777, 0.0520, 0.0787, 0.0178, 0.0419, 0.0448, 0.0019,
          0.0486, 0.0982, 0.0675, 0.0598, 0.0828],
         [0.0055, 0.0223, 0.0021, 0.0447, 0.0527, 0.0410, 0.0101, 0.0292,
          0.0451, 0.0450, 0.0295, 0.0080, 0.0536]]])
batch 1
tensor([[[0.1000, 0.5000, 0.8000, 0.7000, 0.1000, 0.1000, 1.0000, 0.2000,
          0.0696, 0.0474, 0.0423, 0.0736, 0.0602],
         [0.0345, 0.0571, 0.0519, 0.0324, 0.0421, 0.0735, 0.0874, 0.0769,
          0.0395, 0.0184, 0.0786, 0.0174, 0.0891]],

        [[0.0104, 0.0601, 0.0371, 0.0779, 0.0124, 0.0992, 0.0137, 0.0766,
          0.0276, 0.0661, 0.0603, 0.0369, 0.0202],
         [0.1000, 0.7000, 0.1000, 0.0661, 0.0662, 0.0708, 0.0795, 0.0857,
          0.6000,

For each cell choose the predicted bounding box with the higher confidence.

The goal is to achieve `pred.shape == label.shape`.

In [395]:
# extract the predicted classes
# (batch, cells, cells, classes)
pred_classes = pred[..., :CLASSES]

# extract the predicted bounding boxes
# (batch, cells, cells, 5 * b)
pred_boxes = pred[..., CLASSES:]

# place the boxes in their own dimensions
# (batch, cells, cells, b, 5)
pred_boxes = pred_boxes.view((BATCHES, CELLS, CELLS, BOXES, 5))

# extract the probabilities of the boxes
# (batch, cells, cells, b)
pred_probs = pred_boxes[..., 0]

# find the indices of the maximum probabilities
# (batch, cells, cells, 1)
_, indices = torch.max(pred_probs, dim=-1, keepdim=True)

# reshape the indices to match the boxes
# (batch, cells, cells, 1, 5)
indices = indices.unsqueeze(-1).repeat((1, 1, 1, 1, 5))

# gather the boxes with the maximum indices
# pred_boxes[i, j, k, 0, l] <- pred_boxes[i, j, k, indices[i, j, k, l], l]
# pred_boxes[batch, cells, cells, 1, 5]
pred_boxes = torch.gather(pred_boxes, dim=3, index=indices)

# remove the boxes dimension
# pred_boxes[batch, cells, cells, 5]
pred_boxes = pred_boxes.squeeze(3)

In [396]:
# concat the boxes with the classes
# pred_boxes[batch, cells, cells, classes + 5]
pred = torch.concat((pred_classes, pred_boxes), dim=-1)

In [397]:
print('predictions')
for i in range(BATCHES):
	print(f'batch {i}')
	print(pred[i])

predictions
batch 0
tensor([[[0.8000, 0.2000, 0.5000, 0.9000, 0.4500, 0.5500, 0.8500, 0.9500],
         [0.0931, 0.0965, 0.0706, 0.0991, 0.0570, 0.0791, 0.0298, 0.0918]],

        [[0.0863, 0.0777, 0.0520, 0.0787, 0.0178, 0.0419, 0.0448, 0.0019],
         [0.0055, 0.0223, 0.0021, 0.0451, 0.0450, 0.0295, 0.0080, 0.0536]]])
batch 1
tensor([[[0.1000, 0.5000, 0.8000, 0.7000, 0.1000, 0.1000, 1.0000, 0.2000],
         [0.0345, 0.0571, 0.0519, 0.0395, 0.0184, 0.0786, 0.0174, 0.0891]],

        [[0.0104, 0.0601, 0.0371, 0.0779, 0.0124, 0.0992, 0.0137, 0.0766],
         [0.1000, 0.7000, 0.1000, 0.6000, 0.4400, 0.5200, 0.8000, 0.8000]]])


Convert the objects from being relative to a cell to absolute.

In [398]:
def to_absolute(boxes):
	'''
	input:
		boxes: (batch, cells, cells, classes + 5)
	output:
		absolute boxes: (batch, cells^2, classes + 5)
	'''
	# extract the dimensions
	batch, cells_a, cells_b, cp5 = boxes.shape
	classes = cp5 - 5
	
	# absolute cell size (width, height)
	cell_size = torch.Tensor([1 / cells_b, 1 / cells_a])

	# calculate the offset of each cell
	# such that offset[i][j] = [cell's column, cell's row]
	# (cells, cells, 2)
	offsets = torch.Tensor([
		list(zip(range(cells_b), [i] * cells_b))
		for i in range(cells_a)
	])

	# expand to match the size of boxes
	# (1, cells, cells, offset)
	offsets = offsets.unsqueeze(0)

	# multiply the offsets by the cell sizes to get the absolute cell locations
	cell_location = offsets * cell_size

	# un-normalize the objects's locations
	# location <- cell location + cell size * location
	boxes[..., classes + 1: classes + 3] *= cell_size
	boxes[..., classes + 1: classes + 3] += cell_location

	# un-normalize the objects' sizes
	boxes[..., classes + 3: classes + 5] *= cell_size

	# flatten
	# (batch, cells^2, classes + 5)
	return boxes.reshape((batch, cells_a * cells_b, cp5))

In [399]:
# (batch, cells^2, classes + 5)
pred = to_absolute(pred)
label = to_absolute(label)

In [400]:
print('predictions')
for i in range(BATCHES):
	print(f'batch {i}')
	print(pred[i])

predictions
batch 0
tensor([[0.8000, 0.2000, 0.5000, 0.9000, 0.2250, 0.2750, 0.4250, 0.4750],
        [0.0931, 0.0965, 0.0706, 0.0991, 0.5285, 0.0396, 0.0149, 0.0459],
        [0.0863, 0.0777, 0.0520, 0.0787, 0.0089, 0.5209, 0.0224, 0.0009],
        [0.0055, 0.0223, 0.0021, 0.0451, 0.5225, 0.5148, 0.0040, 0.0268]])
batch 1
tensor([[0.1000, 0.5000, 0.8000, 0.7000, 0.0500, 0.0500, 0.5000, 0.1000],
        [0.0345, 0.0571, 0.0519, 0.0395, 0.5092, 0.0393, 0.0087, 0.0446],
        [0.0104, 0.0601, 0.0371, 0.0779, 0.0062, 0.5496, 0.0068, 0.0383],
        [0.1000, 0.7000, 0.1000, 0.6000, 0.7200, 0.7600, 0.4000, 0.4000]])


In [401]:
print('labels')
for i in range(BATCHES):
	print(f'batch {i}')
	print(label[i])

labels
batch 0
tensor([[1.0000, 0.0000, 0.0000, 1.0000, 0.2500, 0.2500, 0.4500, 0.4500],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 1.0000, 0.2550, 0.7450, 0.3000, 0.3000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.0000, 0.0000]])
batch 1
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 1.0000, 0.7500, 0.7500, 0.4000, 0.4000]])


For the labels in each batch, extract the vectors where the confidence is 1.
This operation will convert the label tensor to a list of tensors.
```
(batch, classes + 5) -> list of (classes + 5,)
```

In [402]:
# [(cells^2, classes + 5,)], len = batches
label = [label[i] for i in range(BATCHES)]

# [(x, classes + 5,)], len = batches
label = [l[l[:, CLASSES] > 0.5] for l in label]

In [403]:
label

[tensor([[1.0000, 0.0000, 0.0000, 1.0000, 0.2500, 0.2500, 0.4500, 0.4500],
         [1.0000, 0.0000, 0.0000, 1.0000, 0.2550, 0.7450, 0.3000, 0.3000]]),
 tensor([[0.0000, 1.0000, 0.0000, 1.0000, 0.7500, 0.7500, 0.4000, 0.4000]])]

For the predictions in each batch, sort the boxes from highest to lowest confidence
and just like for the labels, replace the batch dimension with a list.

In [404]:
# [(cells^2, classes + 5,)], len = batches
pred = [pred[i] for i in range(BATCHES)]

In [405]:
for i in range(BATCHES):
	# (cells^2, classes + 5,)
	p = pred[i]

	# (cells^2,)
	p_probs = p[..., CLASSES]

	# (cells^2,)
	indices = torch.argsort(p_probs, descending=True)

	# (cells^2, classes + 5,) - sorted
	pred[i] = p[indices]

In [406]:
pred

[tensor([[0.8000, 0.2000, 0.5000, 0.9000, 0.2250, 0.2750, 0.4250, 0.4750],
         [0.0931, 0.0965, 0.0706, 0.0991, 0.5285, 0.0396, 0.0149, 0.0459],
         [0.0863, 0.0777, 0.0520, 0.0787, 0.0089, 0.5209, 0.0224, 0.0009],
         [0.0055, 0.0223, 0.0021, 0.0451, 0.5225, 0.5148, 0.0040, 0.0268]]),
 tensor([[0.1000, 0.5000, 0.8000, 0.7000, 0.0500, 0.0500, 0.5000, 0.1000],
         [0.1000, 0.7000, 0.1000, 0.6000, 0.7200, 0.7600, 0.4000, 0.4000],
         [0.0104, 0.0601, 0.0371, 0.0779, 0.0062, 0.5496, 0.0068, 0.0383],
         [0.0345, 0.0571, 0.0519, 0.0395, 0.5092, 0.0393, 0.0087, 0.0446]])]

For each batch, calculate the iou matrix such that:
```
mat[i][j] = iou of label box i and predicted box j
```
Then, iterate over all the predicted boxes, assigning a prediction to a target and
calculating if box is a true positive, or a false positive.

In [407]:
def iou_vec(target, boxes):
	'''
	Calculates the IOU of each box in `boxes` with the target.
	input:
		target: (classes + 5,)
		boxes: (x, classes + 5)
	output: the iou
		(x,)
	'''
	# extract the number of classes
	classes = target.shape[0] - 5

	# match the dimensions
	# (1, classes + 5)
	target = target.unsqueeze(0)
	
	# extract the target's location and size
	# (1, 2)
	t_l = target[..., classes + 1: classes + 3]
	t_s = target[..., classes + 3: classes + 5]

	# extract the boxes' locations and sizes
	# (x, 2)
	b_l = boxes[..., classes + 1: classes + 3]
	b_s = boxes[..., classes + 3: classes + 5]

	# extract the target's points - top left and bottom right
	# (1, 2)
	t_tl = t_l - t_s / 2
	t_br = t_l + t_s / 2

	# extract the boxes's points - top left and bottom right
	# (x, 2)
	b_tl = b_l - b_s / 2
	b_br = b_l + b_s / 2

	# x segment shared between the boxes
	# (x,)
	delta_x = (
		torch.minimum(t_br[..., 0], b_br[..., 0]) -
		torch.maximum(t_tl[..., 0], b_tl[..., 0])
	).clamp(0.0, None)

	# y segment shared between the boxes
	# (x,)
	delta_y = (
			torch.minimum(t_br[..., 1], b_br[..., 1]) -
			torch.maximum(t_tl[..., 1], b_tl[..., 1])
	).clamp(0.0, None)

	# intersection and union
	# (x,)
	intersection = delta_x * delta_y
	union = b_s[..., 0] * b_s[..., 1] + t_s[..., 0] * t_s[..., 1] - intersection

	# iou with numerical stability
	return intersection / (union + 1e-5)

In [408]:
def iou_mat(a, b):
	'''
	input: matrices with boxes
		a: (x, classes + 5)
		b: (y, classes + 5)
	output: a matrix such that mat[i][j] = iou(a[i], b[j])
		mat: (x, y)
	'''
	# list of (y,), len = x
	rows = [iou_vec(a[i], b) for i in range(a.shape[0])]
	
	# reshape to list of (1, y), and concat to (x, y)
	return torch.cat([row.unsqueeze(0) for row in rows], dim=0)

In [432]:
def filter_class(t, c):
	'''
	input:
		t: (?, classes + 5)
		c: int
	out:
		filtered: (?, classes + 5) such that argmax(filtered[:classes]) = c
	'''
	classes = t.shape[1] - 5
	
	# extract the classes
	t_classes = t[:, :classes]

	# arg-max over the classes
	idx = torch.argmax(t_classes, dim=-1)
	
	# filter by the arg-max
	filtered = idx == c
	return t[filtered]

def calculate_image_stats(pred, label, iou_threshold, c):
	'''
	input:
		pred: list of (?, classes + 5) - predicted boxes
		label: list of (?, classes + 5) - label boxes
		iou_threshold - predictions share an iou above this threshold count as
			true positives
		c - the class
	output:
		image_stats: list of stats for each item in the batch.
		stats:
			num_labels - number of labels
			stats - list of tuples: ('TP' or 'FP', the confidence of that box)
	'''
	image_stats = []

	for b in range(BATCHES):
		# (x, 5)
		p = filter_class(pred[b], c)
		
		# (y, 5)
		l = filter_class(label[b], c)

		# if there are no positive labels then all boxes are false positives
		if l.shape[0] == 0:
			stats = []
			for i in range(p.shape[0]):
				confidence = p[i][CLASSES].item()
				classification = 'FP'
				stats.append((classification, confidence))
		else:
			# (x, y)
			ious = iou_mat(p, l)

			stats = []
			for i in range(p.shape[0]):
				# (y,)
				label_ious = ious[i]

				# find the label with the maximum IOU
				# (1,)
				val, idx = torch.max(label_ious, dim=0)

				# if the IOU is above the threshold, mark the box as a true positive,
				# and consume the label
				confidence = p[i][CLASSES].item()
				if val > iou_threshold:
					classification = 'TP'
					ious[: idx] = -1
				else:
					classification = 'FP'
				
				stats.append((classification, confidence))

		image_stats.append({
			'num_labels': l.shape[0],
			'stats': stats,
		})
	
	return image_stats

In [433]:
# example
calculate_image_stats(pred, label, 0.5, 0)

[{'num_labels': 2,
  'stats': [('TP', 0.8999999761581421), ('FP', 0.07871577888727188)]},
 {'num_labels': 0, 'stats': []}]

In [411]:
# allocate a container from the stats
iou_c_stats = {}
for i in IOU_THRESHOLDS:
	iou_c_stats[i] = {}
	for j in range(CLASSES):
		iou_c_stats[i][j] = {}

# calculate the precision vs recall graph for all thresholds and for all classes
for iou_threshold in IOU_THRESHOLDS:
	for c in range(CLASSES):
		# combine the stats from all the batches
		num_labels = 0
		stats = []
		for item in calculate_image_stats(pred, label, iou_threshold, c):
			num_labels += item['num_labels']
			stats += item['stats']
		
		# if there are no positive labels, ignore this case
		if num_labels == 0:
			iou_c_stats[iou_threshold][c] = {
				'precision': [],
				'recall': [],
				'confidence': [],
			}
		else:
			# sort by box confidence
			stats.sort(key=lambda s: s[1], reverse=True)

			# calculate precision, recall, confidence
			precision = []
			recall = []
			confidence = []
			tps = 0
			positive_predictions = 0
			for box_m, box_c in stats:
				if box_m == 'TP':
					tps += 1
				
				positive_predictions += 1
				precision.append(tps / positive_predictions)
				recall.append(tps / num_labels)
				confidence.append(box_c)
			
			iou_c_stats[iou_threshold][c] = {
				'precision': precision,
				'recall': recall,
				'confidence': confidence,
			}

In [436]:
def area(xs, ys):
	assert len(xs) == len(ys)
	a = 0
	for i in range(len(xs) - 1):
		delta_x = xs[i + 1] - xs[i]
		y2 = ys[i + 1]
		y1 = ys[i]
		a += delta_x / 2 * (3 * y2 - y1)
	return a

# calculate the mean average precision
map_xs = []
map_ys = []
for iou_threshold in IOU_THRESHOLDS:
	map_xs.append(iou_threshold)
	map_ys.append(sum(
		area(
			iou_c_stats[iou_threshold][c]['recall'],
			iou_c_stats[iou_threshold][c]['precision']
		)
		for c in range(CLASSES)
	) / CLASSES)